In [ ]:
from os import system
import umap
from pylab import *
import pandas as pd
from os import system
import os.path
import platform
import requests
import random
from astropy.io import fits
#from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
import numpy
import scipy
import gc

In [ ]:
DOWNLOAD RVS FLUXES

In [ ]:
system('wget -r --no-parent -nH http://cdn.gea.esac.esa.int/Gaia/gdr3/Spectroscopy/rvs_mean_spectrum')

In [ ]:
system('gunzip -d -r Gaia/gdr3/Spectroscopy/rvs_mean_spectrum')

In [ ]:
import pandas as pd
import os
import numpy as np

def read_rvs_csv(file_path):
    df = pd.read_csv(file_path, skiprows=83)
    return df

def save_ids_to_csv(df, file_path):
    df.to_csv(file_path, columns=['source_id'], index=False)
    
spectra_folder_path = 'Gaia/gdr3/Spectroscopy/rvs_mean_spectrum'
full_df = pd.DataFrame()
for file in os.listdir(spectra_folder_path):
    if file.endswith('csv'):
        print(f'{file}')
        df_temp = read_rvs_csv(os.path.join(spectra_folder_path, file))[['source_id', 'flux']]
        full_df = full_df.append(df_temp, ignore_index=True)
        
        
full_df.to_pickle('rvs_flux_dataframe.pkl')
save_ids_to_csv(full_df, 'rvs_source_ids.csv')

print('DONE')

# for i in range(len(indexes)):
#     with fits.open('/storage/home/yardenei/gaia/fits/' + str(data.apogee_id[indexes[i]])+'.fits') as temp_fits:
#         wl_size=8575
#         vwave = numpy.zeros(wl_size) # vacuum wl
#         awave = numpy.zeros(wl_size) # air wl
#         wl_delta = float(temp_fits[1].header['CDELT1'])
#         wl_logstart = float(temp_fits[1].header['CRVAL1'])
#         start_pix = int(temp_fits[1].header['CRPIX1'])
#         #cur_wl_size = numpy.min([wl_size,int(hdulist[0].header['NWAVE'])])
#         vwave[0] = 10**wl_logstart
#         for j in range (start_pix, wl_size):
#             vwave[j] = vwave[j-1]*(10**wl_delta)
#             awave = vwave / (1.0 +  5.792105E-2/(238.0185E0 - (1.E4/vwave)**2) + 1.67917E-3/( 57.362E0 - (1.E4/vwave)**2))
#         flux1 = temp_fits[1].data.copy()
#         flux2 = temp_fits[2].data.copy()
#         flux3 = temp_fits[3].data.copy()
#     if i==0:
#         df_flux_arr = pd.DataFrame({'pcn':[flux1],'fit':[flux3]})
#     else:
#         df_flux_arr=df_flux_arr.append(pd.DataFrame({'pcn':[flux1],'fit':[flux3]}),ignore_index=True)
#     print(str(i/len(indexes)*100) + '% done')
#     gc.collect()
# fluxes = empty((len(indexes), len(concatenate((arange(246,3274),arange(3585,6080),arange(6344,8335) )))))
# fit = empty((len(indexes), len(concatenate((arange(246,3274),arange(3585,6080),arange(6344,8335) )))))
# #wavelengths = empty((len(indexes), len(concatenate((arange(246,3274),arange(3585,6080),arange(6344,8335) )))))
# for i in arange(len(indexes)):
#     fluxes[i] = concatenate((df_flux_arr.pcn[i][246:3274],df_flux_arr.pcn[i][3585:6080],df_flux_arr.pcn[i][6344:8335]), axis=None)
#     fit[i] = concatenate((df_flux_arr.fit[i][246:3274],df_flux_arr.fit[i][3585:6080],df_flux_arr.fit[i][6344:8335]), axis=None)
#     #wavelengths[i] = concatenate((df_flux_arr.awave[i][246:3274],df_flux_arr.awave[i][3585:6080],df_flux_arr.awave[i][6344:8335]), axis=None)
# ##Replace None values by the mean
# #imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# #fluxes = imp.fit_transform(fluxes)
# #%%

# savez('fluxes.npz', fluxes = fluxes, fit = fit)

In [ ]:
DOWNLOAD ASTROPHYSICAL PARAMETERS

In [ ]:
system('wget -r --no-parent -nH http://cdn.gea.esac.esa.int/Gaia/gdr3/Astrophysical_parameters/astrophysical_parameters')

In [ ]:
system('gunzip -d -r Gaia/gdr3/Astrophysical_parameters/astrophysical_parameters')

In [ ]:
import pandas as pd
import os
import numpy as np

def read_rvs_csv(file_path):
    df = pd.read_csv(file_path, skiprows=1541)
    return df

def save_ids_to_csv(df, file_path):
    df.to_csv(file_path, columns=['source_id'], index=False)
    
spectra_folder_path = 'Gaia/gdr3/Astrophysical_parameters/astrophysical_parameters'
full_df = pd.DataFrame()
for file in os.listdir(spectra_folder_path):
    if file.endswith('csv'):
        print(f'{file}')
        df_temp = read_rvs_csv(os.path.join(spectra_folder_path, file))[['source_id', 'flux']]
        full_df = full_df.append(df_temp, ignore_index=True)
        
        
full_df.to_pickle('astrophysical_parameters_dataframe.pkl')
# save_ids_to_csv(full_df, 'rvs_source_ids.csv')

print('DONE')

In [ ]:
# filter the spectra to the range of 150:2250 and remove objects with nan values
import pandas as pd
import numpy as np
df = pd.read_pickle('rvs_flux_dataframe.pkl')
df_filtered = df.copy()
df_filtered.set_index("source_id", inplace=True)
df.reset_index(inplace=True)
import numpy as np
params = pd.read_csv('gaia_params_lite.csv')
rv = params.loc[:,'radial_velocity']
rv_source_id = params.loc[:,'source_id']
min_lambda = 846
max_lambda = 870
wavelength = np.linspace(min_lambda, max_lambda, len(eval(df['flux'][0].replace('NaN', 'np.nan'))))
c = 299792
from scipy.interpolate import interp1d
ids_list = list()
for i in range(len(df)):
    if np.mod(i,1000)==0:
        print(f'{i/len(df)*100}%')
    try:
        flux = eval(df['flux'][i].values[0].replace('NaN', 'np.nan'))
    except:
        flux = eval(df['flux'][i].replace('NaN', 'np.nan'))
    if np.isnan(flux[150:2250]).any():
        ids_list.append(df['source_id'][i])
df_filtered = df_filtered.drop(labels=ids_list)


print(f'{len(df_filtered)/len(df)*100}% objects left after deletion')
source_ids = np.empty(len(df_filtered))
fluxes = np.empty((len(df_filtered), len(range(150, 2250))))
snr = np.empty(source_ids.shape)
for i in range(len(df_filtered)):
    df_temp = df_filtered.iloc[i]
    source_ids[i] = df_filtered.index[i]
    flux = np.array(eval(df_temp.loc['flux'].replace('NaN', 'np.nan')))
    fluxes[i] = flux[150:2250]
    if np.mod(i, 1000) == 0:
        print(f'creating numpy arrays {i/len(df)*100}%')
np.savez('filtered_nans_fluxes_and_ids.npz', fluxes=fluxes, source_ids=source_ids)

In [ ]:
df = pd.read_pickle('rvs_flux_dataframe.pkl')
source_ids = np.empty(len(df))
fluxes = np.empty((len(df), len(range(15, 2250))))
flux_errors = np.empty(fluxes.shape)
snr = np.empty(source_ids.shape)
for i in range(len(df)):
    source_ids[i] = df.index[i]
    fluxes[i] = np.array(eval(df['flux'][source_ids[i]].replace('NaN', 'np.nan')))[15:2250]
    flux_errors[i] = np.array(eval(df['flux_error'][source_ids[i]].replace('NaN', 'np.nan')))[15:2250]
    snr[i] = np.median(fluxes[i]/flux_errors[i])
    if np.mod(i, 1000) == 0:
        print(f'{i/len(df)*100}%')
np.savez('filtered_fluxes_and_ids.npz', fluxes=fluxes, flux_errors=flux_errors, snr=snr, source_ids=source_ids)


FileNotFoundError: [Errno 2] No such file or directory: 'rvs_flux_dataframe.pkl'

In [ ]:
# from pylab import *
import pandas as pd
import random
import numpy as np

#load fluxes
with np.load('filtered_nans_fluxes_and_ids.npz',allow_pickle=True) as f:
    source_ids = f['source_ids']
    random_indexes = random.sample(range(len(source_ids)), int(len(source_ids)*0.2))
    fluxes = f['fluxes'][random_indexes]
print(f'number of objects in dataset = {len(source_ids)}')
# data = pd.read_csv('JOIN_APOGEE_GAIA.csv')
# indexes = arange(len(data))

#create sample of fakes with same distribution (sampled from actual spectrums)
marginal_dist_values = []
marginal_dist_weights = []
for w in range(len(fluxes[0])):
    h = np.histogram(fluxes[:,w],100)
    values = np.empty(len(h[0]))
    for i in range(len(h[0])):
        values[i] = np.mean((h[1][i], h[1][i+1]))
    values = np.delete(values, np.argwhere(h[0]==0))
    weights = np.delete(h[0],np.argwhere(h[0]==0))
    marginal_dist_values.append(values)
    marginal_dist_weights.append(weights)

num_fakes = int(len(random_indexes)*0.5)
fake_fluxes = np.empty((num_fakes, len(fluxes[0])))

for i in range(num_fakes):
    for w in range(len(fluxes[0])):
        fake_fluxes[i][w] = np.double(random.choices(marginal_dist_values[w], marginal_dist_weights[w]))
    if np.mod(i, 1000) == 0:
        print('creating fake fluxes.. ' + str(i/num_fakes*100) + '% done.')
total_fluxes = np.empty((len(fluxes)+len(fake_fluxes),len(fluxes[0,:])))
total_fluxes[0:len(fluxes)] = fluxes
total_fluxes[len(fluxes):len(total_fluxes)] = fake_fluxes
is_true_flux = np.zeros(len(total_fluxes))
is_true_flux[0:len(fluxes[:,0])] = 1
is_true_flux=is_true_flux.astype(int)

np.savez('rvs_fluxes_with_fakes_for_training.npz', total_fluxes = total_fluxes, is_true_flux = is_true_flux, source_ids=source_ids[random_indexes])